In [1]:
def data_pre_processing():

    # Load echonest file
    echonest_file = './data/echonest.csv'

    #Populate the data dictionary
    names = ['track_id', 'acousticness', 'dancebility', 'energy', 'instrumentalness', 'liveliness', 'speechiness', 'tempo', 'valence', 'artist_name', 'album_name']
    data_dict = pd.read_csv(echonest_file, sep = ',', names = names, usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 14, 15], skiprows = 4)

    #Load the track file
    track_file = './data/raw_tracks.csv'

    #Populate the labels dictionary and tracks dictionary
    names =  ['track_id', 'genres', 'track_title']
    tracks_dict = pd.read_csv(track_file, sep = ',', names = names, usecols = [0,27,37], skiprows = 1)

    #clean up the data
    track_title_list = []
    genres_list = []

    for i in data_dict['track_id']:
        if i in tracks_dict['track_id']:
            index = tracks_dict['track_id'][tracks_dict['track_id'] == i].index[0]
            track_title_list.append(tracks_dict['track_title'][index])

            curr_genre = tracks_dict['genres'][index].split(',')[1].split(':')[1].split("'")[1]
            genres_list.append(curr_genre)

    genre_df = pd.DataFrame({'genres': genres_list})
    title_df = pd.DataFrame({'track_titles': track_title_list})
    new_df = pd.concat([genre_df, title_df], axis = 1)
    data_dict = pd.concat([data_dict, new_df], axis = 1)

  #Create the features dictionary and labels dictionary
    features_dict = data_dict[['acousticness', 'dancebility', 'energy', 'instrumentalness', 'liveliness', 'speechiness', 'tempo','valence']].copy()
    gen_list = np.unique(genre_df)
    indice = random.sample(range(0, gen_list.shape[0]-1), 2)
    labels_dict = {}
    print(gen_list[indice[0]], gen_list[indice[1]])
    for j in data_dict['track_id']:
        
        index = data_dict['track_id'][data_dict['track_id'] == j].index[0] 
        
        if data_dict['genres'][index] == gen_list[indice[0]]:
            a = np.random.choice([0,1], size=(1,), p =[ 1./3, 2./3])
            labels_dict[index] = a[0]
            
        elif data_dict['genres'][index] == gen_list[indice[1]]:
            a = np.random.choice([0,1], size=(1,), p =[ 1./3, 2./3])
            labels_dict[index] = a[0]
            
        else:
            a = np.random.choice([0,1], size=(1,), p =[ 2./3, 1./3])
            labels_dict[index] = a[0]
            
    return data_dict, features_dict, labels_dict

In [ ]:
def print_Recommendations(model_predictions, track_dict, songs):
    User_Recommendations = {}
    count = 0
    t_id, art_name, alb_name, track_titles, genres = [], [], [], [], []
    for i in range(len(model_predictions)):
        if (count < songs):
            if (model_predictions[i] == 1):
                a = track_dict['track_id'][i]
                b = track_dict['artist_name'][i]
                c = track_dict['album_name'][i]
                d = track_dict['track_title'][i]
                e = track_dict['genres'][i]
                if type(a) != float and type(b) != float and type(c) != float and type(d) != float and type(e) != float:
                    t_id.append(a)
                    art_name.append(b)
                    alb_name.append(c)
                    track_titles.append(d)
                    genres.append(e)
                    count+=1
        else:
            User_Recommendations['track_id'] = t_id
            User_Recommendations['artist_name'] = art_name
            User_Recommendations['album_name'] = alb_name
            User_Recommendations['track_title'] = track_titles
            User_Recommendations['genres'] = genres
            break
    
    df = pd.DataFrame.from_dict(User_Recommendations)
    return df